In [15]:
import tkinter as tk
from tkinter import StringVar, filedialog, messagebox
import cv2
from PIL import Image, ImageTk
import numpy as np

root = tk.Tk()
root.geometry("1366x768")
root.title("Lab 02")

text_var = tk.StringVar()
text_var.set("Image: ")

label1 = tk.Label(root,
                 textvariable=text_var,
                 anchor=tk.CENTER,
                 bd=3,
                 font=("Arial", 24, "bold"),
                 fg="black",
                 justify=tk.LEFT,
                )
label1.place(x=500, y=10)

def upload_image():
    global img_cv2, img_tk, modified_img
    file_path = filedialog.askopenfilename()
    if file_path:
        img_cv2 = cv2.imread(file_path)
        img_cv2 = cv2.resize(img_cv2, (400, 300), interpolation=cv2.INTER_LANCZOS4)
        img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk = ImageTk.PhotoImage(img_pil)
        imglabel1.config(image=img_tk)
        imglabel1.image = img_tk
        modified_img = img_cv2  # Set modified_img to the original image initially

def save_modified_image():
    global modified_img
    if modified_img is not None:
        # Ask user for file path to save
        file_path = filedialog.asksaveasfilename(defaultextension=".png",
                                               filetypes=[("PNG files", "*.png"),
                                                          ("JPEG files", "*.jpg"),
                                                          ("All files", "*.*")])
        if file_path:
            cv2.imwrite(file_path, modified_img)
            messagebox.showinfo("Image Saved", f"Image saved as {file_path}")
    else:
        messagebox.showwarning("No Image", "No modified image to save.")

def resize():
    global img_cv2, modified_img
    size_str = size_var.get()
    size = float(size_str)
    if (size > 100 or size < 0):
       messagebox.showwarning("Invalid Input", "Please enter a valid percentage.")
    else:
        
        # Calculate new dimensions
        new_width = int(img_cv2.shape[1] * size/100)
        new_height = int(img_cv2.shape[0] * size/100)

        # Resize the image
        modified_img = cv2.resize(img_cv2, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)

        # Convert to ImageTk.PhotoImage and update imglabel2
        img_rgb = cv2.cvtColor(modified_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_resized = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_resized)
        imglabel2.image = img_tk_resized

def rotate():
    global img_cv2, modified_img
    angle_str = angle_var.get()
    try:
        angle = float(angle_str)
    except ValueError:
        messagebox.showwarning("Invalid Input", "Please enter a valid numeric angle.")
        return
    
    if angle < 0 or angle > 360:
        messagebox.showwarning("Invalid Input", "Please enter a valid numeric angle between 0 and 360.")
        return

    if img_cv2 is not None:
        # Get image dimensions
        (h, w) = img_cv2.shape[:2]
        center = (w / 2, h / 2)

        # Calculate rotation matrix and rotate the image
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated_img = cv2.warpAffine(img_cv2, M, (w, h))

        # Crop the image to remove black borders (Optional)
        # Get the size of the rotated image to determine the bounding box
        cos_angle = np.abs(M[0, 0])
        sin_angle = np.abs(M[0, 1])

        # Compute the new bounding dimensions
        new_w = int((h * sin_angle) + (w * cos_angle))
        new_h = int((h * cos_angle) + (w * sin_angle))

        # Adjust the rotation matrix to account for the change in dimension
        M[0, 2] += (new_w / 2) - center[0]
        M[1, 2] += (new_h / 2) - center[1]

        # Perform the rotation with the adjusted matrix
        rotated_img = cv2.warpAffine(img_cv2, M, (new_w, new_h))

        # Resize the image to original size if needed
        rotated_img = cv2.resize(rotated_img, (w, h))

        # Update modified_img
        modified_img = rotated_img

        # Convert to ImageTk.PhotoImage and update imglabel2
        img_rgb = cv2.cvtColor(modified_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_rotated = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_rotated)
        imglabel2.image = img_tk_rotated
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def invert_colors():
    global img_cv2, modified_img
    if img_cv2 is not None:
        inverted_image = cv2.bitwise_not(img_cv2)
        
        modified_img = inverted_image

        img_rgb = cv2.cvtColor(inverted_image, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_inverted = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_inverted)
        imglabel2.image = img_tk_inverted
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def convert_to_greyscale():
    global img_cv2, modified_img
    if img_cv2 is not None:
        # Convert the image to greyscale
        greyscale_img = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)
        # Convert to 3-channel image for display purposes
        greyscale_img = cv2.cvtColor(greyscale_img, cv2.COLOR_GRAY2RGB)
        
        # Update modified_img
        modified_img = greyscale_img
        
        # Convert to ImageTk.PhotoImage and update imglabel2
        img_pil = Image.fromarray(greyscale_img)
        img_tk_greyscale = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_greyscale)
        imglabel2.image = img_tk_greyscale
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def convert_to_bw():
    global img_cv2, modified_img
    if img_cv2 is not None:
        # Convert the image to greyscale
        greyscale_image = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)

        # apply black and white threshold
        _, bw_img = cv2.threshold(greyscale_image, 127, 255, cv2.THRESH_BINARY)
        
        # Update modified_img
        modified_img = bw_img
        
        # Convert to ImageTk.PhotoImage and update imglabel2
        img_pil = Image.fromarray(bw_img)
        img_tk_bw = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_bw)
        imglabel2.image = img_tk_bw
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def convert_to_color():
    global img_cv2, modified_img
    if img_cv2 is not None:
        # Convert the image to RGB
        RGB_img = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
        
        # Update modified_img
        modified_img = RGB_img
        
        # Convert to ImageTk.PhotoImage and update imglabel2
        img_pil = Image.fromarray(RGB_img)
        img_tk_RGB = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_RGB)
        imglabel2.image = img_tk_RGB
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def segment():
    global img_cv2, modified_img
    if img_cv2 is not None:
        #convert to grayscale
        gray = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)

        #add binary threshold
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        #remove noise using morphological operations
        kernel = np.ones((3, 3), np.uint8)
        opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)

        #determine the sure background area
        sure_bg = cv2.dilate(opening, kernel, iterations=3)

        #Find the sure foreground area
        dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
        _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

        #find the unknown region
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(sure_bg, sure_fg)

        #labelling markers
        _, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        markers[unknown == 255] = 0
        segmented_img = img_cv2.copy()
        markers = cv2.watershed(segmented_img, markers)
        segmented_img[markers == -1] = [0, 0, 255]

        #update modified image
        modified_img = segmented_img

        # Convert to ImageTk.PhotoImage and update imglabel2
        img_rgb = cv2.cvtColor(modified_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_segmented = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_segmented)
        imglabel2.image = img_tk_segmented
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def apply_filter(filter_type):
    global img_cv2, modified_img
    if img_cv2 is not None:
        if filter_type == 'min':
            filtered_img = cv2.erode(img_cv2, np.ones((3, 3), np.uint8))
        elif filter_type == 'max':
            filtered_img = cv2.dilate(img_cv2, np.ones((3, 3), np.uint8))
        elif filter_type == 'median':
            filtered_img = cv2.medianBlur(img_cv2, 3)
        elif filter_type == 'gaussian':
            filtered_img = cv2.GaussianBlur(img_cv2, (3, 3), 1.0)
        else:
            return
        
        modified_img = filtered_img

        img_rgb = cv2.cvtColor(filtered_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_filtered = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_filtered)
        imglabel2.image = img_tk_filtered
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def gamma_correction():
    global img_cv2, modified_img
    if img_cv2 is not None:
        gamma = 1.5  # Example gamma value, can be adjustable
        look_up_table = np.array([((i / 255.0) ** gamma) * 255 for i in range(256)]).astype("uint8")
        corrected_img = cv2.LUT(img_cv2, look_up_table)

        modified_img = corrected_img

        img_rgb = cv2.cvtColor(corrected_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_corrected = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_corrected)
        imglabel2.image = img_tk_corrected
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def histogram_equalization():
    global img_cv2, modified_img
    if img_cv2 is not None:
        img_yuv = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2YUV)
        img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
        equalized_img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

        modified_img = equalized_img

        img_rgb = cv2.cvtColor(equalized_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_equalized = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_equalized)
        imglabel2.image = img_tk_equalized
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def white_balance():
    global img_cv2, modified_img
    if img_cv2 is not None:
        # Convert image to float32 for more precise computation
        img_float = img_cv2.astype(np.float32)

        # Calculate the average color of the image
        avg_color = np.mean(img_float, axis=(0, 1))

        # Scale each channel to make the average color neutral (white)
        white_balance_scale = np.mean(avg_color) / avg_color
        white_balanced_img = img_float * white_balance_scale

        # Clip the values to be in valid range [0, 255] and convert back to uint8
        white_balanced_img = np.clip(white_balanced_img, 0, 255).astype(np.uint8)

        modified_img = white_balanced_img

        img_rgb = cv2.cvtColor(white_balanced_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_white_balanced = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_white_balanced)
        imglabel2.image = img_tk_white_balanced
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

size_var = tk.StringVar()

size_label = tk.Label(root, text="Size (%)", font=("arial", 12, "bold"))
size_entry = tk.Entry(root, textvariable=size_var, font=('calibre', 8, "normal"))
button1 = tk.Button(root, text="Resize Image", command=resize)

button1.place(x=725, y=500)
size_label.place(x=730, y=440)
size_entry.place(x=705, y=470)

button2 = tk.Button(root,
                    command=upload_image,
                    text="Upload Image",
                    bd=3,
                    bg="Lightgray",
                    cursor="hand2",
                    fg="black",
                    font=("Arial", 12),
                    height=1
                  )
button2.place(x=650, y=15)

button3 = tk.Button(root,
                    text="Save Image",
                    command=save_modified_image,
                    bd=3,
                    bg="Lightgray",
                    cursor="hand2",
                    fg="black",
                    font=("Arial", 12),
                    height=1
                  )
button3.place(x=800, y=15)

angle_var = tk.StringVar()

angle_label = tk.Label(root, text="Angle (In Degrees)", font=("arial", 12, "bold"))
angle_entry = tk.Entry(root, textvariable=angle_var, font=('calibre', 8, "normal"))
sub_btn = tk.Button(root, text="Rotate Image", command=rotate)

angle_label.place(x=695, y=550)
angle_entry.place(x=705, y=580)
sub_btn.place(x=725, y=610)

button4 = tk.Button(root, text="Convert to Binary", command=convert_to_bw, bd=3, bg="Lightgray", cursor="hand2", fg="black", font=("Arial", 12), height=1)
button4.place(x=560, y=665)

button5 = tk.Button(root, text="Convert to Color", command=convert_to_color, bd=3, bg="Lightgray", cursor="hand2", fg="black", font=("Arial", 12), height=1)
button5.place(x=703, y=665)

button6 = tk.Button(root, text="Convert to Greyscale", command=convert_to_greyscale, bd=3, bg="Lightgray", cursor="hand2", fg="black", font=("Arial", 12), height=1)
button6.place(x=840, y=665)

button7 = tk.Button(root, text="Invert Colors", command=invert_colors, bd=3, bg="Lightgray", cursor="hand2", fg="black", font=("Arial", 12), height=1)
button7.place(x=900, y=450)

button8 = tk.Button(root, text="Segment Image", command=segment, bd=3, bg="Lightgray", cursor="hand2", fg="black", font=("Arial", 12), height=1)
button8.place(x=500, y=450)

button9 = tk.Button(root, text="Minimum Filter", command=lambda: apply_filter('min'), bd=3, bg="Lightgray", cursor="hand2", fg="black", font=("Arial", 12), height=1)
button9.place(x=200, y=500)

button10 = tk.Button(root, text="Maximum Filter", command=lambda: apply_filter('max'), bd=3, bg="Lightgray", cursor="hand2", fg="black", font=("Arial", 12), height=1)
button10.place(x=200, y=550)

button11 = tk.Button(root, text="Median Filter", command=lambda: apply_filter('median'), bd=3, bg="Lightgray", cursor="hand2", fg="black", font=("Arial", 12), height=1)
button11.place(x=200, y=600)

button12 = tk.Button(root, text="Gaussian Blur Filter", command=lambda: apply_filter('gaussian'), bd=3, bg="Lightgray", cursor="hand2", fg="black", font=("Arial", 12), height=1)
button12.place(x=200, y=650)

button13 = tk.Button(root, text="Gamma Correction", command=gamma_correction, bd=3, bg="Lightgray", cursor="hand2", fg="black", font=("Arial", 12), height=1)
button13.place(x=1200, y=500)

button14 = tk.Button(root, text="Histogram Equalization", command=histogram_equalization, bd=3, bg="Lightgray", cursor="hand2", fg="black", font=("Arial", 12), height=1)
button14.place(x=1200, y=550)

button15 = tk.Button(root, text="White Balance", command=white_balance, bd=3, bg="Lightgray", cursor="hand2", fg="black", font=("Arial", 12), height=1)
button15.place(x=1200, y=600)

imglabel1 = tk.Label(root)
imglabel1.place(x=250, y=125)

imglabel2 = tk.Label(root)
imglabel2.place(x=800, y=125)

root.mainloop()